In [1]:
import os
import time

import pandas as pd

In [2]:
base_data = pd.read_parquet("data/volume-input_E_07-10-2019_1401.parquet.gzip")

In [3]:
base_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11854682 entries, 0 to 11854681
Data columns (total 18 columns):
Datetime       datetime64[ns, UTC]
FUEL           object
Day            int64
Month          int64
HH             float64
Date           datetime64[ns, UTC]
GSP_GROUP      object
Meter_Type     object
Register       object
GSP            object
Shape          float64
TDCV           object
Consumption    float64
d-loss         float64
t-loss         float64
CT_Volume      float64
GSP_Volume     float64
NBP_Volume     float64
dtypes: datetime64[ns, UTC](2), float64(8), int64(2), object(6)
memory usage: 1.7+ GB


In [4]:
no_of_runs = 5

test_dict =[
    {
        'test': 'csv_no_compression',
        'extension' : 'csv',
        'compression' : None,
    },
    {
        'test': 'csv_compression',
        'extension' : 'csv.gzip',
        'compression' : 'gzip',
    },
    {
        'test': 'parquet_no_compression',
        'extension' : 'parquet',
        'compression' : None,
    },
    {
        'test': 'parquet_compression',
        'extension' : 'parquet.gzip',
        'compression' : 'gzip',
    },
]

### Run format tests

In [5]:
results_list = []

for test in test_dict:
    for i in range(no_of_runs):
        print(f'Test: {test["test"]} Run:{i}')
        file_path = f'testing/{test["test"]}_{i}.{test["extension"]}'
        
        if 'csv' in test['test']:
            start = time.time()
            base_data.to_csv(file_path, compression=test['compression'])
            end = time.time()
        else:
            start = time.time()
            base_data.to_parquet(file_path)
            end = time.time()
        write_time = end - start
        
        size = os.path.getsize(file_path)

        if 'csv' in test['test']:
            start = time.time()
            df = pd.read_csv(file_path, compression=test['compression'])
            end = time.time()
        else:
            start = time.time()
            df = pd.read_parquet(file_path)
            end = time.time()
        end = time.time()
        del df
        read_time = end - start

        results = {
            'test' : test['test'],
            'run' : i,
            'write_time' : write_time,
            'read_time' : read_time,
            'file_size' : size,
        }
        results_list.append(results)

Test: csv_no_compression Run:0
Test: csv_no_compression Run:1
Test: csv_no_compression Run:2
Test: csv_no_compression Run:3
Test: csv_no_compression Run:4
Test: csv_compression Run:0
Test: csv_compression Run:1
Test: csv_compression Run:2
Test: csv_compression Run:3
Test: csv_compression Run:4
Test: parquet_no_compression Run:0
Test: parquet_no_compression Run:1
Test: parquet_no_compression Run:2
Test: parquet_no_compression Run:3
Test: parquet_no_compression Run:4
Test: parquet_compressionn Run:0
Test: parquet_compressionn Run:1
Test: parquet_compressionn Run:2
Test: parquet_compressionn Run:3
Test: parquet_compressionn Run:4


In [26]:
results_frame.to_csv("results.csv")
results_frame.to_parquet("results.parquet")

In [27]:
results_frame = pd.read_parquet("results.parquet")

In [28]:
format_dict = {'write_time':'{0:,.1f}s', 'read_time': '{0:,.1f}s', 'file_size': '{0:,.0f}mb'}

table = (
    results_frame.groupby('test')
    .mean()[['write_time', 'read_time', 'file_size']]
    .assign(file_size=lambda x: x['file_size'] / 1024 / 1024)
    .style.format(format_dict)
    .highlight_min(color='lightgreen')
    .highlight_max(color='#cd4f39')
)

table

,write_time,read_time,file_size
test,,,
csv_compression,443.7s,44.0s,193mb
csv_no_compression,345.9s,34.0s,"2,435mb"
parquet_compression,16.0s,9.0s,182mb
parquet_no_compression,15.8s,9.2s,182mb
